In [2]:
import numpy as np
import pandas as pd
from datetime import timedelta, date
import time
import re
import requests
from bs4 import BeautifulSoup
from selenium.webdriver.chrome.service import Service

from selenium import webdriver
# import webbrowser


In [ ]:
##For the single page
# url = requests.get('https://www.yelp.co.uk/search?cflt=burgers&find_loc=Berlin%2C+Germany')
# print(url)
# soup = BeautifulSoup(url.content, 'html.parser')

In [12]:
num = []
for i in range(0,240,10):
    num.append(i)


soups = [] 
for page_num in num:
    url = requests.get(f'https://www.yelp.co.uk/search?find_desc=restaurants+&find_loc=Berlin%2C+Germany&start={page_num}')
    print(url)
    soups.append( BeautifulSoup(url.content, 'html.parser') )
    time.sleep(30)


<Response [200]>
<Response [200]>
<Response [200]>
<Response [200]>
<Response [200]>
<Response [200]>
<Response [200]>
<Response [200]>
<Response [200]>
<Response [200]>
<Response [200]>
<Response [200]>
<Response [200]>
<Response [200]>
<Response [200]>
<Response [200]>
<Response [200]>
<Response [200]>
<Response [200]>
<Response [200]>
<Response [200]>
<Response [200]>
<Response [200]>
<Response [200]>


In [12]:
##storing the soups to avoid scrapping again
hotelsoupdata= pd.DataFrame({'soup':soups})
hotelsoupdata.to_csv('hotelsoupdata.csv', index=False) 

In [13]:
resturant_names2 =[]
ratings2 = []
number_of_rating2 =[]
price_level2 = []
neighbourhood2 = []
type_of_resturant2 = []
for soup in soups:
    #Resturant name 
    htags =soup.find_all("h3",class_="css-kagwww")
    for tags in htags:
        resturant_names2.append(tags.text.replace('\xa0','').split('.')[1])
    #Number of reviews
    rates = soup.find_all('span', class_='reviewCount__09f24__tnBk4 css-chan6m')
    for rate in rates:
        number_of_rating2.append(rate.text)
    #Prices
    princing = soup.select('.css-dzq7l1')
    for price in princing:
        price_output = price.text
        price_level2.append(re.findall('€+',price_output))
    #Location
    neighbour = soup.select('.css-dzq7l1')
    for neighb in neighbour:
        neighb_output= neighb.text
        neighbourhood2.append(re.findall('€+(\w+)',neighb_output))
    #Ratings
    ratings = soup.find_all('div',{'role':'img'})
    for rate in ratings:
        ratings2.append((rate['aria-label']).split(' ')[0])
    #Resturant types
    resturant_types = soup.select('.css-dzq7l1')    
    for types in resturant_types:
        types_output = types.text
        type_of_resturant2.append(types_output.split('€')[0])

        




In [84]:
ratings2[0].split(' ')[0]

'4.5'

In [79]:
len(type_of_resturant2)

240

In [14]:
##For cleaning the price column 
final_data =pd.DataFrame({'Names':resturant_names2, 'Prices':price_level2,'Neighbourhoods':neighbourhood2,'Number_of_reviews':number_of_rating2,'Ratings':ratings2,'Type_of_resturant':type_of_resturant2})
final_data=final_data.mask(final_data.applymap(str).eq('[]'))
final_data=final_data.explode(['Prices'])
final_data=final_data.explode(['Neighbourhoods'])
final_data

,Names,Prices,Neighbourhoods,Number_of_reviews,Ratings,Type_of_resturant
0,Stadtklause,€,Kreuzberg,171,4.5,German
1,Momotaro Tavern,€€,Mitte,60,4.5,BurgersAsian Fusion
2,Aigner Gendarmenmarkt,€€€,Mitte,120,4,GermanAustrian
3,JOSEPH,NaN,NaN,5,4,IsraeliMitte
4,Gaffel Haus Berlin,€€,Mitte,74,4,RhinelandianPubs
...,...,...,...,...,...,...
235,Hugos,€€€€,Tiergarten,37,5,GermanModern European
236,heimlichTreu,€€€,Mitte,8,4.5,Modern European
237,Sabzi,€,Mitte,18,4.5,OrientalTakeaway & Fast FoodMiddle Eastern
238,Cozymazu,€,Wedding,15,4.5,Tea RoomsTaiwanese


In [14]:
#saving into a csv file 
final_data.to_csv('Resturantdata.csv',index=False)

In [2]:
#For the single page
url_amenities = requests.get('https://www.yelp.co.uk/search?cflt=burgers&find_loc=Berlin%2C+Germany')
print(url_amenities)
soup_url_amenitie = BeautifulSoup(url_amenities.content, 'html.parser')

<Response [200]>


In [ ]:
s=Service('chromedriver.exe')
driver = webdriver.Chrome(service=s)

In [32]:
amenities = soup_url_amenitie.find_all('span',class_= 'css-1egxyvc')
result_amenities1=[]
all_nonevales=[]
for amen in amenities:
    try: 
        another_url = amen.a['href']
        url_2 = 'https://www.yelp.co.uk{}'.format(another_url)
        driver.get(url_2)
        time.sleep(2)
        cookies_button = driver.find_element_by_xpath('//*[@id="onetrust-accept-btn-handler"]')
        time.sleep(2)
        cookies_button.click()
        
        # result.append(url_2)
    except:
        # continue
        all_nonevales.append(np.nan)
      
    


In [35]:
another_url

'/biz/peter-pane-berlin-5'

In [34]:
result[1]

'https://www.yelp.co.uk/biz/bbi-berlinburger-international-berlin'

In [5]:
my_amenity = amenities[0].a['href']
the_ur1 = 'https://www.yelp.co.uk{}'.format(my_amenity)
responses = requests.get(the_ur1)
thepage= BeautifulSoup(responses.content,'html.parser')


In [6]:
theamenites = []
available_amenities = thepage.find_all("span",class_="css-1p9ibgf")
for avail in available_amenities:
    theamenites.append(avail.text)
theamenites

['Takes Reservations', 'Offers Takeout', 'Many Vegetarian Options']

In [42]:
available_amenities2 = thepage.find("p",class_="css-1wayfxy")

In [46]:
dtext = available_amenities2.text
dtext

<p class="css-1wayfxy" data-font-weight="semibold">8 More Attributes</p>

In [4]:

# from selenium.webdriver.chrome.service import Service

# from selenium import webdriver
# # import webbrowser
s=Service('chromedriver.exe')
driver = webdriver.Chrome(service=s)
driver.get('https://www.yelp.co.uk/biz/taste-berlin-2?osq=restaurants')
time.sleep(2)
cookies_button = driver.find_element_by_xpath('//*[@id="onetrust-accept-btn-handler"]')
time.sleep(2)
cookies_button.click()
time.sleep(1)
amenitys = driver.find_element_by_xpath('//*[@aria-label="Amenities and More"]') 
amenties_button = amenitys.find_element_by_xpath('.//*[@class=" css-gme0a6"]')#the '.' means find in here
time.sleep(1)
amenties_button.click()
new_soup=BeautifulSoup(driver.page_source)
all_amenities = new_soup.find_all("span",class_="css-1p9ibgf")




C:\Users\T430\AppData\Local\Temp/ipykernel_1908/2326157149.py:9: DeprecationWarning: find_element_by_* commands are deprecated. Please use find_element() instead
  cookies_button = driver.find_element_by_xpath('//*[@id="onetrust-accept-btn-handler"]')
C:\Users\T430\AppData\Local\Temp/ipykernel_1908/2326157149.py:13: DeprecationWarning: find_element_by_* commands are deprecated. Please use find_element() instead
  amenitys = driver.find_element_by_xpath('//*[@aria-label="Amenities and More"]')
C:\Users\T430\anaconda3\envs\strive\lib\site-packages\selenium\webdriver\remote\webelement.py:393: UserWarning: find_element_by_* commands are deprecated. Please use find_element() instead
  warnings.warn("find_element_by_* commands are deprecated. Please use find_element() instead")


In [12]:
allam=[]
for a in all_amenities:
    allam.append(a.text)

allam

['Takes Reservations',
 'Offers Delivery',
 'Offers Takeout',
 'Outdoor Seating',
 'Moderate Noise',
 'Good for Groups',
 'Good For Kids',
 'Waiter Service',
 'Free Wi-Fi',
 'Full Bar',
 'TV']

In [20]:
# amenties_button = driver.find_element_by_id(dtext)
amenitys = driver.find_element_by_xpath('//*[@aria-label="Amenities and More"]') 
amenties_button = amenitys.find_element_by_xpath('.//*[@class=" css-gme0a6"]')#the '.' means find in here
time.sleep(1)
amenties_button.click()


C:\Users\T430\AppData\Local\Temp/ipykernel_15676/84947164.py:2: DeprecationWarning: find_element_by_* commands are deprecated. Please use find_element() instead
  amenitys = driver.find_element_by_xpath('//*[@aria-label="Amenities and More"]')
C:\Users\T430\anaconda3\envs\strive\lib\site-packages\selenium\webdriver\remote\webelement.py:393: UserWarning: find_element_by_* commands are deprecated. Please use find_element() instead
  warnings.warn("find_element_by_* commands are deprecated. Please use find_element() instead")


In [21]:
new_soup=BeautifulSoup(driver.page_source)

In [38]:
do=[]
take = new_soup.find_all("span",class_="css-1p9ibgf")
# for ta in take:
#     do.append(ta.text)

AttributeError: ResultSet object has no attribute 'text'. You're probably treating a list of elements like a single element. Did you call find_all() when you meant to call find()?

In [37]:
do

['Takes Reservations',
 'Offers Delivery',
 'Offers Takeout',
 'Outdoor Seating',
 'Moderate Noise',
 'Good for Groups',
 'Good For Kids',
 'Waiter Service',
 'Free Wi-Fi',
 'Full Bar',
 'TV']

In [27]:

# anenn = []
# # url_2 = 'https://www.yelp.co.uk{}'.format(another_url)
# response = requests.get(url_2)
# pages = BeautifulSoup(response.content,'html.parser')
# featur = soup.find_all('div',class_= 'arrange__09f24__LDfbs gutter-2__09f24__CCmUo layout-wrap__09f24__GEBlv layout-2-units__09f24__PsGVW border-color--default__09f24__NPAKY')
# # features = pages.find_all('section',class_= 'margin-t4__09f24__G0VVf padding-t4__09f24__Y6aGL border--top__09f24__exYYb border-color--default__09f24__NPAKY')
# for fean in featur:
#     featn = fean.find_all("span",class_="css-1p9ibgf")
#     for featsn in featn:
#         anenn.append(featsn.text)


In [38]:
# rest_amenitiess = []
# css_class =  soup.select('.css-1p9ibgf')
# for amenity in css_class:
#     rest_amenitiess.append(amenity.text)


[]